In [1]:
#Author: Ivan Torres-Rodriguez --- IIMAS UNAM
#email: ivantr18@hotmail.com

import numpy as np
import tensorflow as tf
#from tensorflow import keras
import keras
#from tensorflow.keras import backend as K
from keras.layers import *
from keras.layers import Input, Dense
from keras.models import Model
from keras.models import Sequential
from keras.preprocessing import sequence


Using TensorFlow backend.


In [2]:
batch_size = 128
num_classes = 22
epochs = 25
img_rows, img_cols = 1, 1
np.random.seed(5)

In [ ]:
#X=np.load('T22Audio/feat_X.npy')
#Y=np.load('T22Audio/feat_Y.npy')
X=np.load('DIMEX100/Features/feat_X.npy')
Y=np.load('DIMEX100/Features/feat_Y.npy')
#X=np.array(rawX[:,:img_rows * img_cols], dtype='float32')
#Prev=np.load('T22Audio/prevL.npy')
Prev=np.load('DIMEX100/Features/prevL.npy')
#Zeros=X==0
#offset=X.min()
#X=X-X.min()
#X[Zeros]=0
#X=X.reshape(X.shape[0],img_rows, img_cols)
dicY={'a':0,'b':1,'d':2,'e':3,'f':4,'g':5,'i':6,'k':7,'l':8,'m':9,'n':10,'n~':11,'o':12,'p':13,'r':14,'r(':15,'s':16,'t':17,'tS':18,'u':19,'x':20,'Z':21}
dicPrev={'-':-1,'a':0,'b':1,'d':2,'e':3,'f':4,'g':5,'i':6,'k':7,'l':8,'m':9,'n':10,'n~':11,'o':12,'p':13,'r':14,'r(':15,'s':16,'t':17,'tS':18,'u':19,'x':20,'Z':21}
Y=[dicY[key] for key in Y]
Y=np.array(Y, dtype='uint8')
Prev=[dicPrev[key] for key in Prev]
Prev=np.array(Prev, dtype='uint8')
Prev=Prev+1
sizeInp=X.shape[0]
permutation = np.random.permutation(sizeInp)
X=X[permutation]
Y=Y[permutation]
Prev=Prev[permutation]
X=sequence.pad_sequences(X)
#normalizacion
#mean=X.mean()
#std=X.std()
#X=X-mean
#X=X/std
X=X-X.min()
X=X/X.max()
img_cols=X.shape[1]


x_train=X[:int(0.9*sizeInp)]
x_test=X[int(0.9*sizeInp):]
y_train=Y[:int(0.9*sizeInp)]
y_test=Y[int(0.9*sizeInp):]
Prev_te=Prev[int(0.9*sizeInp):]

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
#x_train /= X.max()
#x_test /= X.max()

#Save y_train/test before onehot encoding
np.save('T22Audio/train_features_Y.npy',y_train)
np.save('T22Audio/test_features_Y.npy',y_test)
np.save('T22Audio/test_prev.npy',Prev_te)
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = keras.Sequential()
model.add(keras.layers.Conv2D(100, kernel_size=(3, 3),padding='same' ,activation='relu',input_shape=input_shape))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Conv2D(200, kernel_size=(3, 3), padding='same',activation='relu'))
model.add(keras.layers.Conv2D(400, kernel_size=(3, 3), padding='same',activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

#model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(650, activation='relu'))
#model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(num_classes, activation='softmax'))

In [ ]:
embed_dim = 128
lstm_out = 500
input_shape=(X.shape[1],)

def Conv():
    model_lstm = Sequential()
    model_lstm.add(Embedding(img_cols, embed_dim,input_length = X.shape[1]))
    model_lstm.add(LSTM(lstm_out))
    #print(model_lstm.summary())
    return model_lstm


def FCN():
    model_fc = Sequential()
    model_fc.add(Dense(num_classes, activation='softmax'))
    #input_net = Input(shape=(lstm_out,))  # adapt this if using `channels_first` image data format   
    return model_fc

x = Input(shape=(input_shape))
modelFCN=FCN()
modelConv=Conv()
# make the model:
model = Model(x, modelFCN(modelConv(x)))
print(model.summary())

In [ ]:
#model.compile(keras.optimizers.Adam(lr=0.001),
model.compile(keras.optimizers.Adam(),
              loss=keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

In [ ]:
np.save('T22Audio/test_features_X.npy',modelConv.predict(x_test))
np.save('T22Audio/train_features_X.npy',modelConv.predict(x_train))

ite=0
##Get data
test_X=np.load('T22Audio/test_features_X.npy')
train_X=np.load('T22Audio/train_features_X.npy')
teY=np.load('T22Audio/test_features_Y.npy')
trY=np.load('T22Audio/train_features_Y.npy')



    # The ranges of all the memories that will be trained
sizes = (1, 2, 4, 8, 16, 32, 64, 128, 256, 512)
    # the domain size. The size of the output layer of the network
domain = 650
    # the number of categories
catN=22
    # Maximum value of the features in the train set
    #max_val = train_X.max()

    # Train the different co-domain memories
    
tables = np.zeros((len(sizes), catN, 5), dtype=np.float64)
entropies = np.zeros((len(sizes), int(catN)), dtype=np.float64)


print('Train the different co-domain memories -----',ite)
        #for i, s in enumerate(sizes):
            #list_tables_entropies=get_ams_results1(i, s, domain, train_X, test_X, trY, teY)
list_tables_entropies = Parallel(n_jobs=8, verbose=50)(
    delayed(get_ams_results1)(i, s, domain, train_X, test_X, trY, teY) for i, s in enumerate(sizes))

for i, table, entropy in list_tables_entropies:
    tables[i, :, :] = table
    entropies[i, :] = entropy
    
np.save('T22Audio/tables.npy', tables)
np.save('T22Audio/entropies.npy', entropies)

    # Table columns
    # 0.- Total count
    # 1.- Able to reduce and it is the same number
    # 2.- Able to reduce and it is not the same number
    # 3.- Not able to reduce and it is not the same number
    # 4.- Not able to reduce and it is the same number

    ##########################################################################################

    # Calculate the precision and recall

print('Calculate the precision and recall')
precision = np.zeros((len(sizes), catN+1, 1), dtype=np.float64)
recall = np.zeros((len(sizes), catN+1, 1), dtype=np.float64)

for i, s in enumerate(sizes):
    prec_aux = tables[i, :, 1] / (tables[i, :, 1] + tables[i, :, 2])
    recall_aux = tables[i, :, 1] / tables[i, :, 0]
    precision[i, 0:catN, 0] = prec_aux[:]
    precision[i, catN, 0] = prec_aux.mean()
    recall[i, 0:catN, 0] = recall_aux[:]
    recall[i, catN, 0] = recall_aux.mean()
    

    ######################################################################################

    # Plot of precision and recall with entropies

print('Plot of precision and recall with entropies-----{0}'.format(ite))
average_entropy.append( entropies.mean(axis=1) )
    # Percentage
average_precision.append( precision[:, catN, :] * 100 )
average_recall.append( recall[:, catN, :] * 100 )
    
np.save('average_precision.npy', average_precision)
np.save('average_recall.npy', average_recall)
np.save('average_entropy.npy', average_entropy)
    
print('avg precision: ',average_precision[ite])
print('avg recall: ',average_recall[ite])
print('avg entropy: ',average_entropy[ite])

    # Setting up a colormap that's a simple transtion
cmap = mpl.colors.LinearSegmentedColormap.from_list('mycolors',['cyan','purple'])

    # Using contourf to provide my colorbar info, then clearing the figure
Z = [[0,0],[0,0]]
step = 0.1
levels = np.arange(0.0, 90 + step, step)
CS3 = plt.contourf(Z, levels, cmap=cmap)

plt.clf()


plt.plot(np.arange(0, 100, 10), average_precision[ite], 'r-o', label='Precision')
plt.plot(np.arange(0, 100, 10), average_recall[ite], 'b-s', label='Recall')
plt.xlim(-0.1, 91)
plt.ylim(0, 102)
plt.xticks(np.arange(0, 100, 10), sizes)

plt.xlabel('Range Quantization Levels')
plt.ylabel('Percentage [%]')
plt.legend(loc=4)
plt.grid(True)

entropy_labels = [str(e) for e in np.around(average_entropy[ite], decimals=1)]

cbar = plt.colorbar(CS3, orientation='horizontal')
cbar.set_ticks(np.arange(0, 100, 10))
cbar.ax.set_xticklabels(entropy_labels)
cbar.set_label('Entropy')

plt.savefig('T22Audio/graph_T22_{0}.png'.format(ite), dpi=500)
print('Iteration {0} complete'.format(ite))

In [42]:
X=np.array([[1,2,3,4,5,6],[7,8,9,10,11,12],[13,14,15,16,17,18],[19,20,21,22,23,24]])
Y=np.array([[25,26,27,28,29,30],[31,32,33,34,35,36],[37,38,39,40,41,42],[43,44,45,46,47,48]])

In [43]:
L=[]
X=X.flatten()
Y=Y.flatten()
L.append(X)
L.append(Y)
L=np.array(L)

In [44]:
L.shape

(2, 24)

In [45]:
L=L.reshape(L.shape[0],4,6)

In [53]:
newL=[]
for element in L:
    newL.append(element.T)
newL=np.array(newL)

In [56]:
newL.shape

(2, 6, 4)

In [57]:
test_X=np.load('T22Audio/test_features_X.npy')
train_X=np.load('T22Audio/train_features_X.npy')
teY=np.load('T22Audio/test_features_Y.npy')
trY=np.load('T22Audio/train_features_Y.npy')

In [68]:
train_X.min()

1.1980534e-05